In [10]:
import os, subprocess, re, glob

# ---------------------------------------------------------------------------
# config
SCENE_THRESHOLD   = 0.25   # ffmpeg scene-change sensitivity
MIN_PLAY_SECONDS  = 3      # ignore clips shorter than this
MAX_CLIPS         = 5      # stop after first 5 clips (test run)
SNAP_AT           = 0.5    # first JPEG at 0.5 s into clip
SNAP_STEP         = 0.4    # spacing between JPEGs
NUM_SNAP_FRAMES   = 3
OUT_DIR           = "cuts"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------------------------------------------------------------------
def scene_cuts(video, thr=SCENE_THRESHOLD):
    r = subprocess.run(
        ['ffmpeg','-i',video,'-vf',f"select='gt(scene,{thr})',showinfo",
         '-f','null','-'],
        stderr=subprocess.PIPE, text=True)
    return sorted(float(t) for t in re.findall(r'pts_time:(\d+\.\d+)', r.stderr))

def video_len(video):
    d = subprocess.check_output(
        ['ffprobe','-v','error','-show_entries','format=duration',
         '-of','default=nk=1:nw=1',video], text=True)
    return float(d.strip())

def cut_play(src, start, end, idx, plays_dir):
    out = os.path.join(plays_dir, f'play_{idx:03}.mp4')
    subprocess.run(['ffmpeg','-y','-ss',str(start),'-t',str(end-start),
                    '-i',src,'-c','copy',out],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return out

def snap_frames_ffmpeg(clip, snaps_dir):
    base = os.path.splitext(os.path.basename(clip))[0]
    for i in range(NUM_SNAP_FRAMES):
        t = SNAP_AT + i * SNAP_STEP
        subprocess.run(['ffmpeg','-y','-ss',f'{t}','-i',clip,
                        '-frames:v','1',
                        os.path.join(snaps_dir, f'{base}_snap{i+1}.jpg')],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# ---------------------------------------------------------------------------
clip_total = 0
for video in glob.glob("*.mp4"):
    if clip_total >= MAX_CLIPS:
        break

    base = os.path.splitext(os.path.basename(video))[0]
    match_dir  = os.path.join(OUT_DIR, base)
    plays_dir  = os.path.join(match_dir, "plays")
    snaps_dir  = os.path.join(match_dir, "snaps")
    os.makedirs(plays_dir,  exist_ok=True)
    os.makedirs(snaps_dir, exist_ok=True)

    cuts  = scene_cuts(video)
    times = [0.0] + cuts + [video_len(video)]

    play_idx = 1
    for s, e in zip(times[:-1], times[1:]):
        if e - s < MIN_PLAY_SECONDS:
            continue
        clip_total += 1
        clip_path = cut_play(video, s, e, play_idx, plays_dir)
        snap_frames_ffmpeg(clip_path, snaps_dir)
        play_idx += 1
        if clip_total >= MAX_CLIPS:
            break

print(f"Done: {clip_total} clip(s) + {clip_total * NUM_SNAP_FRAMES} JPEG(s). ✅")

Done: 5 clip(s) + 15 JPEG(s). ✅
